In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras import Model, optimizers, callbacks, regularizers, layers
from keras.layers import Dense, Convolution1D, Dropout, TimeDistributed, Input
from ss_functions_alt import *
from ss_pred_classes import *

path = 'C:/Users/vinicius/Downloads/data/'

In [2]:
x_train1, x_train2, y_train, lengths= get_data2(path+'training/','list.txt',padding_x=False)
x_train1, x_train2, lengths = np.array(x_train1), np.array(x_train2), np.array(lengths)
y_train = np.array(y_train)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1200,) + inhomogeneous part.

In [3]:
x_train, y_train, lengths= get_data3(path+'training/','list.txt', padding_x=True,padding_y=True)
x_train, lengths, y_train = np.array(x_train), np.array(lengths), np.array(y_train)

In [13]:
x_train, y_train= get_data(path+'training/','list.txt', padding_x=True,padding_y=True)
x_train, y_train = np.array(x_train), np.array(y_train)

In [2]:
def get_data3(path, file, encode_y=True, padding_x=True, padding_y=True, test=False):
    x = []
    y = []
    lengths = []
    with open(path+file, 'r') as sample_file:  # add some stuff to check?
        for line in sample_file:
            line = line.rstrip()
            if test:
                line = line.replace(':', '_')
            pssm, sequence = parse_pssm(path, line, padding=padding_x)
            lengths.append(len(sequence))
            sequence_hot = aa_onehot_encoding(sequence, padding=padding_x)
            features = np.concatenate((sequence_hot, pssm), axis=1)
            x.append(features)

            dssp = parse_dssp(path, line).replace('-', 'C')
            if encode_y:
                dssp = ss_onehot_encoding(dssp, padding=padding_y)

            y.append(dssp)
    return x, y, lengths

In [4]:
weights = {}
mask = []
train_mask = []
get, _ = get_data(path+'training/','list.txt', padding_x=False)
for i in get:
    a = np.zeros((800))
    a[:len(i)] = 1
    mask.append(tf.expand_dims(a, axis=1))
mask = np.array(mask)
train_mask = np.array(train_mask)

In [14]:
stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_modeltest.keras',
    monitor='val_weighted_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
decay=0.98
def lr_decay(epoch, lr):
    return (decay**epoch)*lr

lr_schedule = callbacks.LearningRateScheduler(lr_decay)

def get_model():
    inputs = Input((800,41))
    input_mask = Input((800,1))

    X = DeepInception_block()(inputs)

    X = DeepInception_block()(X)

    # X = layers.CuDNNLSTM(512,return_sequences=True)(X)

    X = layers.multiply([X, input_mask])
    X = layers.Masking(mask_value=0)(X)
    X = layers.LSTM(512,return_sequences=True)(X)
    # X = Convolution1D(41, 19, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
    
    # X = layers.add([X, inputs])

    X = DeepInception_block()(X)
    X = Convolution1D(100, 19, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)

    # X = layers.add([X, inputs])
    # X = layers.multiply([X, input_mask])
    # X = layers.Masking(mask_value=0)(X)
    # X = layers.LSTM(512,return_sequences=True)(X)
    X = layers.multiply([X, input_mask])
    X = layers.Masking(mask_value=0)(X)
    X = layers.LSTM(512,return_sequences=True)(X)
    X = TimeDistributed(Dense(256, activation='relu'))(X)
    X = Dropout(0.4)(X)
    Y = TimeDistributed(Dense(3, activation='softmax'))(X)

    model = Model(inputs=[inputs, input_mask], outputs=[Y])
    return model

In [15]:
model = get_model()

opt = optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              weighted_metrics='accuracy',
              metrics=['accuracy'])

In [110]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_49 (InputLayer)          [(None, 800, 41)]    0           []                               
                                                                                                  
 deep_inception_block_56 (DeepI  (None, 800, 369)    234274      ['input_49[0][0]']               
 nception_block)                                                                                  
                                                                                                  
 deep_inception_block_57 (DeepI  (None, 800, 369)    359242      ['deep_inception_block_56[0][0]']
 nception_block)                                                                                  
                                                                                           

In [16]:
with tf.device('/GPU:1'):
    history = model.fit([x_train,mask], y_train,
                        epochs=100,
                        batch_size=12,
                        validation_split=0.2,
                        sample_weight=mask,
                        callbacks=[stop_monitor_loss, checkpoint, lr_schedule])
    
# 3rd block - negligeable improvement
# mask - did not work

Epoch 1/100
80/80 [==============================] - ETA: 0s - loss: 1.3952 - accuracy: 0.4130 - weighted_accuracy: 0.4130
Epoch 1: val_weighted_accuracy improved from -inf to 0.42518, saving model to .\ss_pred_modeltest.keras
80/80 [==============================] - 141s 783ms/step - loss: 1.3952 - accuracy: 0.4130 - weighted_accuracy: 0.4130 - val_loss: 0.6132 - val_accuracy: 0.4252 - val_weighted_accuracy: 0.4252 - lr: 0.0050
Epoch 2/100
80/80 [==============================] - ETA: 0s - loss: 0.3454 - accuracy: 0.4234 - weighted_accuracy: 0.4234
Epoch 2: val_weighted_accuracy did not improve from 0.42518
80/80 [==============================] - 48s 596ms/step - loss: 0.3454 - accuracy: 0.4234 - weighted_accuracy: 0.4234 - val_loss: 0.2951 - val_accuracy: 0.4252 - val_weighted_accuracy: 0.4252 - lr: 0.0049
Epoch 3/100
80/80 [==============================] - ETA: 0s - loss: 0.2551 - accuracy: 0.4302 - weighted_accuracy: 0.4302
Epoch 3: val_weighted_accuracy improved from 0.42518 to 

In [17]:
test_model = keras.models.load_model('ss_pred_modeltest.keras', 
                                     custom_objects={
                                         'inception_conv': inception_conv,
                                         'InceptionNet_paper': InceptionNet_paper,
                                         'DeepInception_block': DeepInception_block,
                                         'truncated_accuracy': truncated_accuracy
                                     })
# test_model.summary()

In [11]:
x_test, y_test = get_data(path+'blindTest/','list.txt',encode_y=False, padding_x=True, padding_y=False, test=True)
x_test = np.array(x_test)
input_mask_test = []
get, _ = get_data(path+'blindTest/','list.txt', padding_x=False, test=True)
for i in get:
    a = np.zeros((800))
    a[:len(i)] = 1
    input_mask_test.append(tf.expand_dims(a, axis=1))
input_mask_test = np.array(input_mask_test) 

In [19]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
predictions_hot = test_model.predict((x_test, input_mask_test))
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)

total = 0
TP = 0
for prediction, truth in zip(predictions, y_test):
    for i, ss in enumerate(truth):
        total +=1
        if ss==prediction[i]:
            TP+=1

accuracy = TP/total
print(accuracy)

11/11 [==============================] - 2s 171ms/step
0.7657718552851809


In [39]:
x_test1, x_test2, y_test, lengths = get_data2(path+'blindTest/','list.txt',encode_y=False, padding_x=True, padding_y=False, test=True)
input_mask_test = []
get, _ = get_data(path+'blindTest/','list.txt', padding_x=False, test=True)
for i in get:
    a = np.zeros((800))
    a[:len(i)] = 1
    input_mask_test.append(tf.expand_dims(a, axis=1))


In [ ]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
predictions_hot = test_model.predict([np.array(x_test1),np.array(x_test2), np.array(input_mask_test)])
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)

total = 0
TP = 0
for prediction, truth in zip(predictions, y_test[:200]):
    for i, ss in enumerate(truth):
        total +=1
        if ss==prediction[i]:
            TP+=1

accuracy = TP/total
print(accuracy)

ValueError: in user code:

    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 216, in assert_

input:
1. one hot encoded sequence
2. PSSM

Model:
1D convolutional neural network

output:
multiclass classification - dense layer with relu activaiton - 3?

validation metric - accuray + model specific measures

soruces:
https://www.csbj.org/article/S2001-0370(22)00506-2/fulltext
